In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import autograd, optim
import torch.utils.data as Data

In [2]:
df = pd.read_csv('Home_price.csv')
df.head()

,Date,area,price
0,01/02/2020,3600,100345
1,02/02/2020,4056,345432
2,03/02/2020,6543,445678
3,04/02/2020,5676,344555
4,05/02/2020,4322,112344


In [3]:
x = df.drop(['price','Date'],axis=1).to_numpy()
y = df['price'].to_numpy()

In [4]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [5]:
x_train, y_train, x_test, y_test = map(torch.tensor, (x_train, y_train, x_test, y_test))

In [6]:
x_train = x_train.float()
y_train = y_train.float()
x_test = x_test.float()
y_test = y_test.float()

In [7]:
input_size = x_train.size()[-1]
hidden_size = 4
output = 1
learning_rate = 0.001

In [14]:
class Model(nn.Module):
    def __init__(self, input_size,hidden_size,output):
        super().__init__()
        self.h1 = nn.Linear(input_size,hidden_size)
        self.h2 = nn.Linear(hidden_size,output)

    def forward(self,x):
        x = self.h1(x)
        x = F.relu(x)
        x = self.h2(x)
        return x

In [15]:
model = Model(input_size=input_size,hidden_size=hidden_size,output=output)

In [16]:
optimization = optim.Adam(params=model.parameters(),lr=learning_rate)
loss_func = nn.MSELoss()

In [21]:
model.train()
for epoch in range(5):
    predictions = model(x_train)
    loss = loss_func(predictions.squeeze(),y_train)
    print(f'loss {loss}, epoch {epoch+1}')

    optimization.zero_grad() # clear gradients for next train
    loss.backward() # backpropagation, compute gradients
    optimization.step() # apply gradients


loss 137165021184.0, epoch 1
loss 137160081408.0, epoch 2
loss 137155158016.0, epoch 3
loss 137150251008.0, epoch 4
loss 137145335808.0, epoch 5


In [23]:
model.eval()
y_pred = model(x_test)
after_train = loss_func(y_pred.squeeze(), y_test)
print('Test loss after Training' , after_train.item()) 

Test loss after Training 14478065664.0


In [24]:
new_x = autograd.Variable(torch.Tensor([[3600.0]]))
y_pred = model(new_x)
print("predicted Y value: ", y_pred.data[0][0])

predicted Y value:  tensor(-1363.2412)


In [25]:
model = nn.Sequential(
 nn.Linear(input_size, hidden_size),
 nn.ReLU(),
 nn.Linear(hidden_size, output),
 )

In [27]:
optimization = optim.Adam(params=model.parameters(),lr=learning_rate)
loss_func = nn.MSELoss()

In [31]:
Batch_size = 64
torch_dataset = Data.TensorDataset(x_train,y_train)
loader = Data.DataLoader(dataset = torch_dataset,batch_size=Batch_size,shuffle=True)

In [32]:
for epoch in range(5):
    for step,(batch_x,batch_y) in enumerate(loader):
         b_x = autograd.Variable(batch_x)
         b_y = autograd.Variable(batch_y)

         prediction = model(b_x)
         loss = loss_func(prediction,b_y)
         model.zero_grad()# clear gradients for next train
         loss.backward() # backpropagation, compute gradients
         optimization.step() # apply gradients
         print(f'loss {loss}, epoch {epoch}')

loss 136395440128.0, epoch 0
loss 136390107136.0, epoch 1
loss 136384765952.0, epoch 2
loss 136379424768.0, epoch 3
loss 136374083584.0, epoch 4


/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [33]:
df = pd.read_csv('insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [34]:
x = df.drop('bought_insurance',axis=1).to_numpy()
y = df['bought_insurance'].to_numpy()
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [35]:
x_train, y_train, x_test, y_test = map(torch.tensor,
(x_train, y_train, x_test, y_test))
x_train = x_train.float()
y_train = y_train.long()
x_test = x_test.float()
y_test = y_test.long()

In [36]:
input_size = x_train.size()[-1]
hidden_size = 4
output = 2
learning_rate = 0.001

In [37]:
class Model(nn.Module):
 def __init__(self, input_size,hidden_size,output):
     super(Model,self).__init__()
     self.h1 = nn.Linear(input_size,hidden_size)
     self.h2 = nn.Linear(hidden_size,output)

 def forward(self,x):
     x = self.h1(x)
     x = F.relu(x)
     x = self.h2(x)
     x = F.sigmoid(x)
     return x

In [38]:
model = Model(input_size=input_size,hidden_size=hidden_size,output=output)
optimization = optim.Adam(params=model.parameters(),lr=learning_rate)
loss_func = nn.CrossEntropyLoss()

In [39]:
model.train()
for epoch in range(5):
 prediction = model(x_train.requires_grad_())
 loss = loss_func(prediction.squeeze(),y_train)

 optimization.zero_grad()# clear gradients for next train
 loss.backward() # backpropagation, compute gradients
 optimization.step() # apply gradients
 print(f'loss {loss}, epoch {epoch+1}')


loss 0.6691502928733826, epoch 1
loss 0.6683945059776306, epoch 2
loss 0.6676867604255676, epoch 3
loss 0.6670227646827698, epoch 4
loss 0.6663864254951477, epoch 5


/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [40]:
model.eval()
y_pred = model(x_test)
after_train = loss_func(y_pred.squeeze(), y_test)
print('Test loss after Training' , after_train.item())

Test loss after Training 0.7990232110023499


In [41]:
new_x = autograd.Variable(torch.Tensor([[90,0]]))
y_pred = model(new_x)
print("predicted Y value: ", y_pred.data[0][0])

predicted Y value:  tensor(6.6627e-05)
